In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import os

N_STATES = 5   # 1维世界的宽度 0:買進,1:買進平倉,2:賣出,3:賣出平倉, 4:無動作
ACTIONS = ['action','close-avg','deal','amp_cover','vol_cover']     # 探索者的可用动作
EPSILON = 0.9   # 贪婪度 greedy
ALPHA = 0.1     # 学习率
GAMMA = 0.9    # 奖励递减值
MAX_EPISODES = 13   # 最大回合数
FRESH_TIME = 0.3    # 移动间隔时间
global inPortfolio
inPortfolio = False # 是否持倉中
global priceAtPurchase #損益
priceAtPurchase = 0


In [2]:
direct=os.path.abspath('../data')
filelist = os.listdir('../data')
file = filelist[-1]
print(file)
dayticks = pd.read_csv(direct+'\\'+file,header=None,names=['ndatetime','nbid','nask','close','volume','deal'])
dayticks['ndatetime'] = pd.to_datetime(dayticks['ndatetime'], format='%Y-%m-%d %H:%M:%S.%f')
dayticks.sort_values(by=['ndatetime'],ascending=True)
dayticks=dayticks[(dayticks.ndatetime.dt.hour>=8) & (dayticks.ndatetime.dt.hour<15)]
dayticks.index = dayticks.ndatetime
Candledf=dayticks['close'].resample('1min',closed='right').ohlc()
tmpdf=dayticks['volume'].resample('1min').sum()
Candledf=pd.concat([Candledf,tmpdf],axis=1)
del tmpdf
tmpdf=dayticks['deal'].resample('1min').sum()
Candledf=pd.concat([Candledf,tmpdf],axis=1)
del tmpdf
Candledf['dealcumsum']=Candledf['deal'].cumsum()
del Candledf['deal']
Candledf.rename(columns={'dealcumsum':'deal'},inplace=True)
Candledf=Candledf.rename_axis('ndatetime').reset_index()
Candledf['ndatetime'] = pd.to_datetime(Candledf['ndatetime'], format='%Y-%m-%d %H:%M:%S.%f')
Candledf.sort_values(by=['ndatetime'],ascending=True)
tmpline=Candledf.close.cumsum()
Candledf['Avg_line'] = tmpline.apply(lambda x: x/(tmpline[tmpline==x].index[0]+1))
del tmpline
Candledf['amplitude'] = Candledf.high - Candledf.low
Candledf['close_Avg'] = Candledf.close - Candledf.Avg_line
global CandleData
CandleData = Candledf[['ndatetime','close_Avg','deal','amplitude','volume','close']]
print(CandleData.tail())
global Total_Trade
Total_Trade = CandleData.shape[0]
Start_Run_Trade = 3



Ticks2021-11-26.txt
     close_Avg  deal  amplitude  volume  close
295 -46.233108 -3892          5     553  17342
296 -44.084175 -3785          4     443  17344
297 -46.926174 -3598          6     617  17341
298 -44.775920 -3507          4     615  17343
299 -45.623333 -3510          9    1191  17342


In [ ]:
# https://blog.csdn.net/joinquantdata/article/details/87254611?spm=1001.2101.3001.6650.6&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-6.no_search_link&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-6.no_search_link
class Market:
    def __init__(self,data) -> None:
        self.action_list = ['Buy','Sell','Wait'] # 買進、賣出、觀望
        self.actions = len(self.action_list)
        self.data = data # 1分鐘 K線資料
        self.time = self.data.at[0,'ndatetime']

        pass